In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch 
from torch import nn
from torch.utils.data import TensorDataset,Dataset,DataLoader,random_split
import datetime
import pandas as pd

# prepare data

In [2]:
with open('config.npy', 'rb') as f:
    config = np.load(f)
with open('J.npy', 'rb') as f:
    J = np.load(f)
with open('energy.npy', 'rb') as f:
    # energy 都是负的，在这里取 正值
    energy = -np.load(f)

In [3]:
data = TensorDataset(torch.tensor(J).float(),torch.tensor(energy).reshape(len(energy),1).float())

n_train = int(len(data)*0.8)
n_valid = len(data) - n_train
ds_train,ds_valid = random_split(data,[n_train,n_valid])

dl_train,dl_valid = DataLoader(ds_train,batch_size = 8),DataLoader(ds_valid,batch_size = 8)


In [4]:
for features,labels in dl_train:
    print(features.shape)
    print(labels.shape)
    break

torch.Size([8, 32])
torch.Size([8, 1])


# prepare model

In [5]:
class Net(nn.Module): 
    def __init__(self):
        super(Net, self).__init__()
        self.linear1 = nn.Linear(32,16)
        self.lrelu1 = nn.LeakyReLU()
        self.linear2 = nn.Linear(16,1)
        self.lrelu2 = nn.LeakyReLU()
        
    def forward(self,x):
        x = self.linear1(x)
#         y = self.lrelu1(x)
        x = self.linear2(x)
        y = self.lrelu2(x)
        return y
        
model = Net()

In [6]:
from torchkeras import summary
summary(model,input_shape= (1000,32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1             [-1, 1000, 16]             528
            Linear-2              [-1, 1000, 1]              17
         LeakyReLU-3              [-1, 1000, 1]               0
Total params: 545
Trainable params: 545
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.122070
Forward/backward pass size (MB): 0.137329
Params size (MB): 0.002079
Estimated Total Size (MB): 0.261478
----------------------------------------------------------------


In [7]:
model.optimizer = torch.optim.SGD(model.parameters(),lr = 0.01)
model.loss_func = torch.nn.MSELoss()
model.metric_func = lambda y_pred,y_true: torch.mean(1-torch.abs(y_true-y_pred))
model.metric_name = "auc"

In [8]:
def train_step(model,features,labels):
    
    # 训练模式，dropout层发生作用
    model.train()
    
    # 梯度清零
    model.optimizer.zero_grad()
    
    # 正向传播求损失
    predictions = model(features)
    loss = model.loss_func(predictions,labels)
    metric = model.metric_func(predictions,labels)

    # 反向传播求梯度
    loss.backward()
    model.optimizer.step()

    return loss.item(),metric.item()

def valid_step(model,features,labels):
    
    # 预测模式，dropout层不发生作用
    model.eval()
    # 关闭梯度计算
    with torch.no_grad():
        predictions = model(features)
        loss = model.loss_func(predictions,labels)
        metric = model.metric_func(predictions,labels)
    
    return loss.item(), metric.item()


# 测试train_step效果
features,labels = next(iter(dl_train))
labels = labels.reshape(8,1)
train_step(model,features,labels)

(155.8196258544922, -11.406691551208496)

In [9]:
labels.reshape(8,1).shape

torch.Size([8, 1])

In [10]:
def train_model(model,epochs,dl_train,dl_valid,log_step_freq):

    metric_name = model.metric_name
    print("Start Training...")
    nowtime = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    print("=========="*8 + "%s"%nowtime)
    df = pd.DataFrame([], columns=['EPOCH','loss','auc','val_loss','val_auc'])
    for epoch in range(1,epochs+1):  

        # 1，训练循环-------------------------------------------------
        loss_sum = 0.0
        metric_sum = 0.0
        step = 1

        for step, (features,labels) in enumerate(dl_train, 1):

            loss,metric = train_step(model,features,labels)

            # 打印batch级别日志
            loss_sum += loss
            metric_sum += metric
            if step%log_step_freq == 0:   
                print(("[step = %d] loss: %.3f, "+metric_name+": %.3f") %
                      (step, loss_sum/step, metric_sum/step))

        # 2，验证循环-------------------------------------------------
        val_loss_sum = 0.0
        val_metric_sum = 0.0
        val_step = 1

        for val_step, (features,labels) in enumerate(dl_valid, 1):

            val_loss,val_metric = valid_step(model,features,labels)

            val_loss_sum += val_loss
            val_metric_sum += val_metric

        # 3，记录日志-------------------------------------------------
        info = (epoch, loss_sum/step, metric_sum/step, 
                val_loss_sum/val_step, val_metric_sum/val_step)
        df = df.append(pd.DataFrame([info], columns=['EPOCH','loss','auc','val_loss','val_auc']),ignore_index=True)
        # 打印epoch级别日志
        print(("\nEPOCH = %d, loss = %.3f,"+ metric_name + \
              "  = %.3f, val_loss = %.3f, "+"val_"+ metric_name+" = %.3f") 
              %info)
        nowtime = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        print("\n"+"=========="*8 + "%s"%nowtime)

    print('Finished Training...')
    
    return df

In [11]:
epochs = 100

history = train_model(model,epochs,dl_train,dl_valid,log_step_freq = 50)

Start Training...
================================================================================2021-10-06 10:18:37
[step = 50] loss: 28.217, auc: -2.552
[step = 100] loss: 15.207, auc: -1.374

EPOCH = 1, loss = 15.207,auc  = -1.374, val_loss = 1.538, val_auc = 0.062

================================================================================2021-10-06 10:18:37
[step = 50] loss: 2.386, auc: -0.230
[step = 100] loss: 2.275, auc: -0.208

EPOCH = 2, loss = 2.275,auc  = -0.208, val_loss = 1.523, val_auc = 0.068

================================================================================2021-10-06 10:18:37
[step = 50] loss: 2.368, auc: -0.226
[step = 100] loss: 2.257, auc: -0.204

EPOCH = 3, loss = 2.257,auc  = -0.204, val_loss = 1.511, val_auc = 0.073

================================================================================2021-10-06 10:18:37
[step = 50] loss: 2.353, auc: -0.222
[step = 100] loss: 2.242, auc: -0.200

EPOCH = 4, loss = 2.242,auc  = -0.200, val_loss = 1.5

[step = 100] loss: 2.106, auc: -0.164

EPOCH = 34, loss = 2.106,auc  = -0.164, val_loss = 1.422, val_auc = 0.108

================================================================================2021-10-06 10:18:38
[step = 50] loss: 2.214, auc: -0.188
[step = 100] loss: 2.104, auc: -0.164

EPOCH = 35, loss = 2.104,auc  = -0.164, val_loss = 1.421, val_auc = 0.108

================================================================================2021-10-06 10:18:38
[step = 50] loss: 2.213, auc: -0.188
[step = 100] loss: 2.103, auc: -0.163

EPOCH = 36, loss = 2.103,auc  = -0.163, val_loss = 1.421, val_auc = 0.109

================================================================================2021-10-06 10:18:38
[step = 50] loss: 2.211, auc: -0.188
[step = 100] loss: 2.101, auc: -0.163

EPOCH = 37, loss = 2.101,auc  = -0.163, val_loss = 1.420, val_auc = 0.109

================================================================================2021-10-06 10:18:38
[step = 50] loss: 2.210, auc: -0.

[step = 100] loss: 2.070, auc: -0.153

EPOCH = 69, loss = 2.070,auc  = -0.153, val_loss = 1.415, val_auc = 0.112

================================================================================2021-10-06 10:18:39
[step = 50] loss: 2.179, auc: -0.178
[step = 100] loss: 2.070, auc: -0.153

EPOCH = 70, loss = 2.070,auc  = -0.153, val_loss = 1.415, val_auc = 0.112

================================================================================2021-10-06 10:18:39
[step = 50] loss: 2.179, auc: -0.177
[step = 100] loss: 2.069, auc: -0.153

EPOCH = 71, loss = 2.069,auc  = -0.153, val_loss = 1.415, val_auc = 0.112

================================================================================2021-10-06 10:18:39
[step = 50] loss: 2.178, auc: -0.177
[step = 100] loss: 2.068, auc: -0.153

EPOCH = 72, loss = 2.068,auc  = -0.153, val_loss = 1.415, val_auc = 0.112

================================================================================2021-10-06 10:18:39
[step = 50] loss: 2.177, auc: -0.

In [12]:
history

,EPOCH,loss,auc,val_loss,val_auc
0,1,15.207211,-1.373539,1.537990,0.062256
1,2,2.274932,-0.207974,1.523182,0.067548
2,3,2.257295,-0.203666,1.510666,0.072541
3,4,2.242060,-0.200017,1.499946,0.076870
4,5,2.228789,-0.196763,1.490718,0.080427
...,...,...,...,...,...
95,96,2.051395,-0.148192,1.414417,0.112534
96,97,2.050735,-0.148021,1.414423,0.112554
97,98,2.050079,-0.147850,1.414432,0.112565
98,99,2.049424,-0.147680,1.414441,0.112576


In [13]:
# 测试model效果
features_test,labels_test = next(iter(dl_valid))
# labels = labels.reshape(8,1)
predictions_test = model(features_test)

In [14]:
predictions_test

tensor([[11.8668],
        [12.2453],
        [12.1601],
        [12.2160],
        [11.9624],
        [11.9592],
        [12.1191],
        [11.9097]], grad_fn=<LeakyReluBackward0>)

In [15]:
labels_test

tensor([[12.9806],
        [13.6311],
        [10.5856],
        [11.0930],
        [10.9182],
        [10.0491],
        [12.1610],
        [10.0896]])